In [ ]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import OneCycleLR, StepLR, LinearLR
from data.preprocessing import *
from data.data_utils import *
from models.dae import *
from utils import *

In [ ]:
input_size = 20
hidden_size = 40
architecture = ()
rnn_num_layers = 1
sequence_length_hour = 96  #last 5 days
num_heads = [2, 10]
bidirectional = True
dict_values = ['dst_kyoto', 'kp_gfz']
time_steps = 0 #after how time steps you want to inference

In [ ]:
device = get_default_device()

In [ ]:
device

device(type='cuda')

In [ ]:
arch = 'LSTM'
model = to_device(LSTMDenoisingAutoEncoder(input_size, hidden_size, rnn_num_layers, bidirectional=bidirectional, num_heads=num_heads, architecture=architecture), device)

In [ ]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = False)
l1_sample = l1_sample.resample('60min').mean()
l2_sample = l2_sample.resample('60min').mean()

c:\Users\jorge\Documents\SMFGF-SpaceApps\data\preprocessing.py:360: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  today_kp = kp[day][0:8]


In [ ]:
hour_kp_dataset = MainToSingleTarget(l1_sample, dst, sequence_length_hour, 1, hour = True, sep = False, target_mode = 'dst_kyoto', l2_df = l2_sample, time_step_ahead = 0, dae = True, multiclass = False)

In [ ]:
batch_loss = []
batch_r2 =[]
for sample in hour_kp_dataset:
    l1, l2 = sample
    batch_loss.append(F.mse_loss(l1,l2))
    batch_r2.append(r2(l1, l2))

torch.stack(batch_loss).mean(), torch.stack(batch_r2).mean()

In [ ]:
batch_loss = sorted([(i, x.item()) for i, x in enumerate(batch_loss)], key=lambda x: x[1])

In [ ]:
fault_train_idx = [i for i,_ in batch_loss[-6000:]]
non_faulty_idx = [i for i, _ in batch_loss[: 4000]]
test_idx = [i for i, _ in batch_loss[4000:-6000]]

SyntaxError: invalid syntax (2325063223.py, line 1)

In [ ]:
dae_train = hour_kp_dataset[[fault_train_idx, non_faulty_idx]]
dae_cross = hour_kp_dataset[test_idx]

In [ ]:
#Test:20% training: 80%
batch_size = 25  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(dae_train, batch_size, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(dae_cross, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [ ]:
history = []

In [ ]:
##hyperparameters
epochs = 100
max_lr = 3e-3
weigth_decay = 1e-8
grad_clip = 1e-3
opt_func = Adam
#lr_sched = OneCycleLR  
lr_sched = LinearLR
start_factor = 1
end_factor = 0.1
steps = epochs
gamma = 0.999
weights = None
encoder_forcing = None
#opt_func = RMSprop

In [ ]:
history += model.fit(epochs, max_lr, train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func, lr_sched, start_factor, end_factor, steps, gamma, weights, encoder_forcing)

KeyboardInterrupt: 

In [ ]:
import os
name = f'{hidden_size}_{rnn_num_layers}_{time_steps}_{arch}'
os.makedirs('dae', exist_ok=True)
torch.save(model, f'dae/{name}.pt')
try:     
    os.remove(f'dae/{name}.csv')
except FileNotFoundError:   
    pass
with open(f'dae/{name}.csv', 'w') as file:
    file.write(','.join(map(str,list(history[-1].values()))))